In [13]:
#!pip3 install twilio

In [14]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [15]:
query = 'Santiago'
api_key = API_KEY_WAPI

#url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima = 'http://api.weatherapi.com/v1/history.json?key=' + api_key +'&q=Santiago&dt=2023-06-23'
url_clima

'http://api.weatherapi.com/v1/history.json?key=0d00f1f6e67347bf89f182241232907&q=Santiago&dt=2023-06-23'

In [16]:
response = requests.get(url_clima).json()

In [17]:
response

{'location': {'name': 'Santiago',
  'region': 'Region Metropolitana',
  'country': 'Chile',
  'lat': -33.45,
  'lon': -70.67,
  'tz_id': 'America/Santiago',
  'localtime_epoch': 1690660730,
  'localtime': '2023-07-29 15:58'},
 'forecast': {'forecastday': [{'date': '2023-06-23',
    'date_epoch': 1687478400,
    'day': {'maxtemp_c': 14.8,
     'maxtemp_f': 58.6,
     'mintemp_c': 8.4,
     'mintemp_f': 47.1,
     'avgtemp_c': 11.9,
     'avgtemp_f': 53.4,
     'maxwind_mph': 8.3,
     'maxwind_kph': 13.3,
     'totalprecip_mm': 30.0,
     'totalprecip_in': 1.18,
     'avgvis_km': 6.5,
     'avgvis_miles': 4.0,
     'avghumidity': 74.0,
     'condition': {'text': 'Moderate rain',
      'icon': '//cdn.weatherapi.com/weather/64x64/day/302.png',
      'code': 1189},
     'uv': 3.0},
    'astro': {'sunrise': '07:47 AM',
     'sunset': '05:43 PM',
     'moonrise': '11:42 AM',
     'moonset': '10:45 PM',
     'moon_phase': 'Waxing Crescent',
     'moon_illumination': '20'},
    'hour': [{'time

In [18]:
response.keys()

dict_keys(['location', 'forecast'])

In [19]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [20]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [21]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1687492800,
 'time': '2023-06-23 00:00',
 'temp_c': 14.6,
 'temp_f': 58.3,
 'is_day': 0,
 'condition': {'text': 'Patchy rain possible',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
  'code': 1063},
 'wind_mph': 4.3,
 'wind_kph': 6.8,
 'wind_degree': 48,
 'wind_dir': 'NE',
 'pressure_mb': 1018.0,
 'pressure_in': 30.07,
 'precip_mm': 0.1,
 'precip_in': 0.0,
 'humidity': 66,
 'cloud': 80,
 'feelslike_c': 14.6,
 'feelslike_f': 58.3,
 'windchill_c': 14.6,
 'windchill_f': 58.3,
 'heatindex_c': 14.6,
 'heatindex_f': 58.3,
 'dewpoint_c': 8.3,
 'dewpoint_f': 46.9,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 5.4,
 'gust_kph': 8.6,
 'uv': 1.0}

# Dataframe

In [22]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [23]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<00:00, 87381.33it/s]


In [24]:
datos[0]

('2023-06-23', 0, 'Patchy rain possible', 14.6, 0, 0)

In [25]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-06-23,0,Patchy rain possible,14.6,0,0
1,2023-06-23,1,Patchy rain possible,14.6,0,0
2,2023-06-23,2,Patchy light drizzle,14.2,0,0
3,2023-06-23,3,Patchy rain possible,14.4,0,0
4,2023-06-23,4,Patchy light drizzle,14.1,0,0
5,2023-06-23,5,Light rain,13.4,0,0
6,2023-06-23,6,Light drizzle,13.7,0,0
7,2023-06-23,7,Light drizzle,13.8,0,0
8,2023-06-23,8,Moderate rain,13.8,0,0
9,2023-06-23,9,Light rain,13.7,0,0


In [26]:
df[df['Temperatura']<10]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
14,2023-06-23,14,Moderate rain,9.6,0,0
15,2023-06-23,15,Light rain,9.2,0,0
16,2023-06-23,16,Light rain,9.4,0,0
17,2023-06-23,17,Moderate rain,8.8,0,0
18,2023-06-23,18,Light rain,8.7,0,0
19,2023-06-23,19,Light rain,9.1,0,0
20,2023-06-23,20,Moderate rain,8.4,0,0
21,2023-06-23,21,Patchy light drizzle,8.5,0,0
22,2023-06-23,22,Patchy light drizzle,9.3,0,0
23,2023-06-23,23,Patchy light drizzle,8.8,0,0


In [27]:
df_rain =  df[df['Temperatura']<10]
df_rain = df_rain[['Hora','Condicion','Temperatura']]
df_rain.set_index('Hora', inplace = True)

In [28]:
df_rain

,Condicion,Temperatura
Hora,,
14,Moderate rain,9.6
15,Light rain,9.2
16,Light rain,9.4
17,Moderate rain,8.8
18,Light rain,8.7
19,Light rain,9.1
20,Moderate rain,8.4
21,Patchy light drizzle,8.5
22,Patchy light drizzle,9.3


In [29]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-06-23 en Santiago es : \n\n\n                  Condicion  Temperatura\nHora                                   \n14           Moderate rain          9.6\n15              Light rain          9.2\n16              Light rain          9.4\n17           Moderate rain          8.8\n18              Light rain          8.7\n19              Light rain          9.1\n20           Moderate rain          8.4\n21    Patchy light drizzle          8.5\n22    Patchy light drizzle          9.3\n23    Patchy light drizzle          8.8'

In [30]:
PHONE_NUMBER

'+1 762 422 4006'

# Mensaje SMS desde Twilio

In [ ]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+569 ____'
                 )

print('Mensaje Enviado ' + message.sid)